In [2]:
import pandas as pd

# Sample DataFrame
data = {'VENDOR_FACILITY': [None, None, '252', '905', None],
        'VENDOR_NUMBER': ['445932', '447709', '008571', '008571', '10000044'],
        'BIC_VENDOR_NUMBER': [None, None, '008571', '008571', None],
        'VENDOR_HOME_LOC_DEST': ['251', '252', None, None, None]}

df = pd.DataFrame(data)

# # Fill missing values in VENDOR_FACILITY with values from VENDOR_HOME_LOC_DEST
# df['VENDOR_FACILITY'] = df['VENDOR_FACILITY'].fillna(df['VENDOR_HOME_LOC_DEST'])

# print("Modified DataFrame:")
# # print(df)
df

,VENDOR_FACILITY,VENDOR_NUMBER,BIC_VENDOR_NUMBER,VENDOR_HOME_LOC_DEST
0,None,445932,None,251
1,None,447709,None,252
2,252,008571,008571,None
3,905,008571,008571,None
4,None,10000044,None,None


In [20]:

# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle
# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

In [ ]:
# Extract values from the Pandas Series
vendor_number = row['VENDOR_NUMBER']
vendor_facility = row['VENDOR_FACILITY']
cs_vend_site_id = row['CS_VEND_SITE_ID']
request_id = row['request_id']

# Use parameterized query
post_cur.execute(
    '''
    INSERT INTO cspoms.XXPO_VENDOR_MASTER (
        CS_VENDOR_ID,
        VENDOR_NUMBER,
        VENDOR_FACILITY,
        CS_VEND_SITE_ID,
        REQUEST_ID,
        CREATION_DATE,
        LAST_UPDATE_DATE,
        CREATED_BY,
        LAST_UPDATED_BY
    ) 
    VALUES (
        nextval('cs_vendor_id_seq'),
        %s,
        %s,
        %s,
        %s,
        current_timestamp,
        current_timestamp,
        '-1',
        '-1'
    ) 
    ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET 
        VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
        VENDOR_FACILITY = EXCLUDED.VENDOR_FACILITY,
        ...
    RETURNING CS_VENDOR_ID
    ''',
    (vendor_number, vendor_facility, cs_vend_site_id, request_id)
)


In [21]:
post_cur.execute('''INSERT INTO XXPO_VENDOR_EDI_INFO (CS_VENDOR_ID,
                VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID,
                CREATION_DATE,
                LAST_UPDATE_DATE,
                CREATED_BY,
                LAST_UPDATED_BY
                ) 
                VALUES (%s, COALESCE(%s, EXCLUDED.VENDOR_CONTINUITY_REPLEN_FLG), %s, current_timestamp, current_timestamp, '-1', '-1') 
                ON CONFLICT (CS_VENDOR_ID) 
                DO UPDATE SET VENDOR_CONTINUITY_REPLEN_FLG = EXCLUDED.VENDOR_CONTINUITY_REPLEN_FLG,
                REQUEST_ID = EXCLUDED.REQUEST_ID,
                LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                ''',
                (generated_c1, your_vendor_cont_replen_flg, your_request_id))

post_conn.commit()



UndefinedTable: missing FROM-clause entry for table "excluded"
LINE 9:                 VALUES (620, COALESCE(NULL, EXCLUDED.VENDOR_...
                                                    ^
